# Classification Tree

1. [Classification Tree](#classification-tree)
    - [gini impurity and entropy](#gini-impurity-and-entropy)
    - [implement decision tree classifier from scratch](#implement-decision-tree-classifer-from-scratch)
1. [Resources](#resources)

## Classification Tree

In the decision_tree_regressor notebook we discussed using the information gain to search for the best split.

\begin{align}
\text{information gain}=H - \frac{n_l}{n}H_l - \frac{n_r}{n}H_p 
\end{align}

Recall that for regression tree, we used the variance as the homogeniety measure.

For classification tree, common homogeneity measures for classification tree are gini impurity and entropy.

### Gini Impurity and Entropy

\begin{align}
\text{gini impurity} & = \sum_k p_k(1-p_k) = 1 - \sum_k p_k^2 \\
\text{entropy} & = - \sum_k p_k * \log p_k
\end{align}

Let's think this through using a binary classification example where a node has 100 samples. 

- If the node is perfectly homogenous (all 100 samples are of the same class). The gini impurity is $0 * 1 - 1 * 0 = 0$. 
- If the node has 90 0s and 10 1s, the gini impurity is $0.9 * 0.1 + 0.1 * 0.9 = 0.18$
- If the node has 50 0s and 50 1s, the gini impurity is $0.5 * 0.5 + 0.5 * 0.5 = 0.50$

Using p as the probability of class 0, the gini impurity can be rewritten to

\begin{align}
L_{\text{gini}} & = p(1-p) + (1-p)p = 2(p-p^2) \\
\frac{dL}{dp} & = 2(1-2p) = 0 \\
p & = \frac{1}{2}
\end{align}

Similarly, the entropy (or log loss) is also maximized when the node has a 50/50 split. 

\begin{align}
L_{\text{entropy}} & = - p\log{p} - (1-p)\log(1-p) \\
\frac{dL}{dp} & = - (\frac{p}{p} + \log{p}) - (-\frac{1-p}{1-p} -\log(1-p)) \\
\frac{dL}{dp} & = \log(1-p) - \log{p} = 0 \\
p & = \frac{1}{2}
\end{align}

We can also see it in the plot below.

In [1]:
import numpy as np
import plotly.express as px
import pandas as pd


p_0 = np.arange(0.01, 1, 0.01)
p_1 = 1 - p_0
gi = 1 - p_0 ** 2 - p_1 ** 2
e = - p_0 * np.log2(p_0)-p_1 * np.log2(p_1)

df = pd.DataFrame({"ratio of class 0": p_0, "gini impurity": gi, "entropy": e, })
px.line(df, x="ratio of class 0", y=["gini impurity", "entropy"], title="Binary Classification Homogeneity")

## Implement Decision Tree Classifer from Scratch

In [2]:
from dataclasses import dataclass

import numpy as np


def gini_impurity(y) -> float:
    p_k = np.array([(y == k).mean() for k in np.unique(y)])
    return p_k.T @ (1 - p_k)


def entropy(y) -> float:
    p_k = np.array([(y == k).mean() for k in np.unique(y)])
    return - p_k.T @ np.log(p_k)


@dataclass
class Node:
    # attributes for non-leaf nodes
    left_child: "Node" = None
    right_child: "Node" = None
    feature: int = None
    threshold: float = None

    # attributes for leaf nodes
    # this is an array of p_k
    value: np.ndarray = None

    def is_leaf(self):
        return self.value is not None


class DTClassifier:
    def __init__(self, criterion="gini", max_depth=3, min_samples_split=2, min_samples_leaf=1, max_features=None, min_information_gain=0):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.max_features = max_features
        self.min_information_gain = min_information_gain

        if criterion == "gini":
            self.impurity_func = gini_impurity
        elif criterion == "entropy":
            self.impurity_func = entropy
        else:
            raise Exception("criterio must be one of ['gini', 'entropy']")
    
    def predict(self, X):
        p = self.predict_proba(X)
        return np.argmax(p, axis=1)

    def predict_proba(self, X):
        return np.array([self._predict_row(x, self.root) for x in X])
    
    def _predict_row(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict_row(x, node.left_child)
        else:
            return self._predict_row(x, node.right_child)
    
    def fit(self, X, y):
        self.n_categories = len(np.unique(y))
        self.root = self._grow_tree(X, y)
    
    def _grow_tree(self, X, y, depth=1):
        # check if stopping criteria is met
        if depth > self.max_depth or len(np.unique(y)) == 1 or len(y) < self.min_samples_split:
            return Node(value=np.bincount(y, minlength=self.n_categories) / len(y))
        
        # find best split that maximize information gain
        best_feature, best_threshold = self._get_best_split(X, y)
        if best_feature is None:
            return Node(value=np.bincount(y, minlength=self.n_categories) / len(y))
        
        # check if the split is legal
        X_left, X_right, y_left, y_right = self._split(X, y, best_feature, best_threshold, return_X=True)
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            return Node(value=np.bincount(y, minlength=self.n_categories) / len(y))
        
        # perform the split
        left_node = self._grow_tree(X_left, y_left, depth+1)
        right_node = self._grow_tree(X_right, y_right, depth+1)
        return Node(left_child=left_node, right_child=right_node, feature=best_feature, threshold=best_threshold)
    

    def _get_best_split(self, X, y):
        m, n = X.shape
        if self.max_features is None or self.max_features > n:
            max_features = n
        else:
            max_features = self.max_features

        H_p = self.impurity_func(y)

        best_ig = - np.inf
        best_feature = None
        best_threshold = None

        for feature in np.random.choice(n, max_features, replace=False):
            thresholds = np.unique(X[:, feature])[1:]
            for threshold in thresholds:
                y_left, y_right = self._split(X, y, feature, threshold)
                H_l = self.impurity_func(y_left)
                H_r = self.impurity_func(y_right)
                ig = H_p - len(y_left) / m * H_l - len(y_right) / m * H_r

                if ig > best_ig:
                    best_ig = ig
                    best_feature = feature
                    best_threshold = threshold
        
        if best_ig < self.min_information_gain:
            return None, None
        
        return best_feature, best_threshold
    

    def _split(self, X, y, feature, threshold, return_X=False):
        left_mask = X[:, feature] < threshold
        right_mask = X[:, feature] >= threshold
        
        if return_X:
            return X[left_mask], X[right_mask], y[left_mask], y[right_mask]
        return y[left_mask], y[right_mask]
    

    def _print_node(self, node, indent):
        if node.is_leaf():
            print(f"{' '*indent}leaf values: {node.value}")
        else:
            print(f"{' '*indent}feature: {node.feature}, threshold: {node.threshold}")
            self._print_node(node.left_child, indent+1)
            self._print_node(node.right_child, indent+1)

    def print_tree(self):
        self._print_node(self.root, 0)

We'll use the algorithm implemented above on the Iris dataset, which presents a multiclass classification problem.

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X = data.data
y = data.target
print(data.keys())
print(data.feature_names)
print(data.target_names)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [4]:
dtc = DTClassifier(max_features=4, max_depth=2)
dtc.fit(X_train, y_train)
dtc.print_tree()

feature: 3, threshold: 1.0
 leaf values: [1. 0. 0.]
 feature: 2, threshold: 4.8
  leaf values: [0.         0.97297297 0.02702703]
  leaf values: [0.         0.11627907 0.88372093]


In [5]:
y_pred = dtc.predict(X_test)
y_pred

array([1, 0, 2, 1, 2, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [6]:
dtc.predict_proba(X_test)

array([[0.        , 0.97297297, 0.02702703],
       [1.        , 0.        , 0.        ],
       [0.        , 0.11627907, 0.88372093],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.11627907, 0.88372093],
       [1.        , 0.        , 0.        ],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.11627907, 0.88372093],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.11627907, 0.88372093],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.11627907, 0.88372093],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.97297297, 0.02702703],
       [0.        , 0.11627907, 0.88372093],
       [1.        , 0.        , 0.        ],
       [0.        , 0.11627907, 0.88372093],
       [1.

In [7]:
from sklearn.metrics import classification_report


print(classification_report(y_test, y_pred, target_names=data.target_names))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.89      0.94         9
   virginica       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



# Resources

- [scikit-learn tree mathematical formation](https://scikit-learn.org/stable/modules/tree.html#tree-mathematical-formulation)